In [30]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [61]:
import pickle
train_path = '/content/drive/MyDrive/ssne-recursive/train.pkl'
with open(train_path, 'rb') as f:
    data = pickle.load(f)

`data` jest postaci: `{(array([float, float, ..., float]), int), ...}`, gdzi tablica floatów to utwór, a int to oznaczenie kompozytora

In [62]:
print(f"Length: {len(data)}, Type: {type(data)}")
print(f"Length[0]: {len(data[0])}")
print(data[0])
print(len(data[0][0]))
print(data[0][0][0])

Length: 2939, Type: <class 'list'>
Length[0]: 2
(array([ -1.,  -1.,  -1., ...,  78.,  40., 144.]), 0)
4756
-1.0


In [63]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import numpy as np

In [64]:
BATCH_SIZE = 32
INPUT_SIZE = 1
NUM_LAYERS = 1
HIDDEN_SIZE = 5
NUM_CLASSES = 5

In [65]:
import math

min_val = math.inf
max_val = -math.inf

for x in data:
    for elt in x[0]:
        if elt < min_val:
            min_val = elt
        if elt > max_val:
            max_val = elt

for i in range(len(data)):
    for j in range(len(data[i][0])):
        data[i][0][j] = np.float32((data[i][0][j] - min_val) / max_val)

In [66]:
from sklearn.model_selection import train_test_split

tensor_data = [(torch.tensor(x[0], dtype=torch.float32), x[1]) for x in data]
train_data, test_data = train_test_split(tensor_data, test_size=0.2, random_state=42)

In [67]:
def collate_fn(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    max_length = max(x_lens)

    padded_xx = [torch.cat([x, torch.zeros(max_length - len(x))]) for x in xx]

    return torch.stack(padded_xx), torch.tensor(yy)

In [68]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [69]:
composers = {0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}

In [76]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(
            input_size=input_size, hidden_size=hidden_size, num_layers=num_layers
        )
        self.fc = nn.Linear(hidden_size, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden

    def forward(self, x, hidden):
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.rnn(x, hidden)
        out = all_outputs[-1]
        x = self.fc(out)
        return x, hidden

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = RNNClassifier(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(device)
model

cuda


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [73]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

for epoch in range(101):
    for x, targets in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device).long()
        hidden = model.init_hidden(x.size(0)).to(device)
        preds, last_hidden = model(x, hidden)
        # preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [75]:
import matplotlib.pyplot as plt

def evaluate(model, test_loader):
    model.eval()  # Tryb ewaluacji (wyłącza dropout, batchnorm itp.)
    total_correct = 0
    total_samples = 0
    loss_values = []

    with torch.no_grad():  # Wyłączanie obliczeń gradientów
        for x, targets in test_loader:
            x = x.to(device).unsqueeze(2)
            targets = targets.to(device)
            hidden = model.init_hidden(x.size(0)).to(device)
            preds, _ = model(x, hidden)
            preds = preds.squeeze(1)

            loss = loss_fun(preds, targets)
            loss_values.append(loss.item())

            _, predicted = torch.max(preds, dim=1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)

    accuracy = total_correct / total_samples * 100
    avg_loss = sum(loss_values) / len(loss_values)

    print(f"Test Accuracy: {accuracy:.2f}%, Test Loss: {avg_loss:.3f}")

    return loss_values, accuracy

# Przeprowadzamy test
loss_values, test_accuracy = evaluate(model, test_loader)

# Wizualizacja wyników
epochs = list(range(1, len(loss_values) + 1))

plt.figure(figsize=(8, 5))
plt.plot(epochs, loss_values, label="Test Loss", color="red")
plt.xlabel("Batch Number")
plt.ylabel("Loss Value")
plt.title("Test Loss Over Batches")
plt.legend()
plt.grid()
plt.show()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
